In [3]:
import cv2
import numpy
import os
import matplotlib.pyplot as plt
import random
import pickle
import numpy as np






# preprocessing , creating fetaures with flags , then we will turn values into numpy arrays and save as pickle files

Picture_Path = [r"C:\Users\agboo\Downloads\Pnemounia Prediction\chest_xray\train"]

Pnemounia_Folders = ["NORMAL","PNEUMONIA"] # we will let pnemounia be one 

# pnemounia icludes virus bound pnemounia and bacterial bound penmounia

Data = []

def normalize_array(arr, new_min, new_max):
    # Find the minimum and maximum values of the array
    old_min = np.min(arr)
    old_max = np.max(arr)
    
    # Apply the normalization formula
    normalized_arr = new_min + (arr - old_min) * (new_max - new_min) / (old_max - old_min)
    return normalized_arr



def Training_Data(Train,row = 250 , col = 250):
    for i in range(2):
        Folder = os.path.join(Picture_Path[0],Pnemounia_Folders[i])
        for img in os.listdir(Folder):
            try:
                img_path = os.path.join(Folder,img)
                new_img = cv2.imread(img_path , cv2.IMREAD_COLOR)
                new_img = cv2.resize(new_img,(row,col))
                Train.append([new_img,i]) # appending the array version of the img along with the flag
            except:
                print("Error - Corrupted File")
        # now we will create two numpy arrays and save the fetaures in them

    random.shuffle(Train) # this way we are avoiding the data being biased to one value

    Images = []
    Flags = []

    for key , flag in Train:
        Images.append(normalize_array(key , 0 , 1))
        Flags.append(flag)

    Images = numpy.array(Images).reshape(-1,row,col,3) # we only need one since we are working with X-rays therfore grayscaled data
    Flags = numpy.array(Flags)

    
    pickle_out = open("Images_Nov.pickle" , "wb")
    pickle.dump(Images , pickle_out)
    pickle_out.close()

    pickle_out = open("Flags_Nov.pickle", "wb")
    pickle.dump(Flags , pickle_out)
    pickle_out.close()

    # this ends our pre-processing for our supervised learning model.
       
Training_Data(Data)
print(len(Data))

5216


In [4]:
import tensorflow
import keras
import numpy as np
import collections
import pickle
from tensorflow.keras.layers import Conv2D , RandomZoom , Activation , Flatten , Dense , MaxPooling2D , AveragePooling2D , Dropout

# we want our model to have a toplogy with a graph like this , conider this graph to be a DAG
edges = [[0,3],[1,3],[2,4],[3,5],[4,5],[5,6],[5,7],[6,8],[7,8]]

# 3 input /conv layers 
# 2  pooling layers
#  1 flatten layers
#  2 dense layers
#  1 output

# we can define paths in the graph like this

seen = set() # global set
    
        
def builder(edges):
    # build adgecency list
    adj = collections.defaultdict(list)
    for start , end in edges:
        adj[start].append(end)
    return adj

def path_finder(adj_list,node,dst):
    seen.add(node)
    if node == dst:
        return seen
    
    for end in adj_list[node]:
        if end not in seen and path_finder(adj_list,end,dst):
            return seen
    return False

# now we must open the file

Images = pickle.load(open("Images_Nov.pickle" ,"rb"))
Flags = pickle.load(open("Flags_Nov.pickle" , "rb"))


       
# tried input normalization to speed up training process but ran into issues 11/3/24
# i think I will just train with the full shape , however this will make training time slower

# creating input nodes
# our input shape is 1250  x 800 , we specified this in the pre-processing X 3 , this is because since its a picture we are working with RGB


# we will do this with diffrent  nodes so our values will be diffrent

img_input_shape = keras.Input(Images.shape[1:])



convo_layer0 = Conv2D(32 , (3,3))

convo_layer1 = Conv2D(28 , (2,2))

convo_layer2 = Conv2D(64 , (3,3))

# then we will link the maxpooling layers 

pooling3 = MaxPooling2D(pool_size = (3,3))

pooling4 = MaxPooling2D(pool_size = (2,2))


# first conv - > pooling

A = convo_layer0(img_input_shape)
a = pooling3(A)

B = convo_layer1(img_input_shape)
b = pooling3(B)

C = convo_layer2(img_input_shape)
c = pooling4(C)

# Flatten Layer

flatten5 = Flatten()

#  pooling - > flatten

d = flatten5(b)
e = flatten5(c)

# Dense 

Dense6 = Dense(48 , activation = "relu" , use_bias = False)

Dense7 = Dense(48 , activation = "relu" , use_bias = True)

# flatten -> Dense

f = Dense6(d)
g = Dense7(e)


# Technical Output Layer

Dense8 = Dense(1 , activation = "sigmoid") # since this a binary classification

# dense - > Output

h = Dense8(f)
i = Dense8(g)


# we will then form all the paths 
# this will give us 6 distinct paths


model_one = keras.Model(inputs = img_input_shape , outputs = h , name = "Model1")
model_two = keras.Model(inputs = img_input_shape , outputs = i , name = "Model2")

from tensorflow.keras.callbacks import TensorBoard

Board_Name = "Comparison"

tensor = TensorBoard(log_dir = 'logs/{}'.format(Board_Name))

model_one.compile(loss = "binary_crossentropy" , optimizer = "adam" , metrics = ["accuracy"]) # compile both models
model_two.compile(loss = "binary_crossentropy" , optimizer = "adam" , metrics = ["accuracy"])



model_one.fit(Images , Flags , batch_size = 25 , epochs = 25 , validation_split = 0.2 , callbacks = [tensor])
model_two.fit(Images , Flags, batch_size = 25 , epochs = 25 , validation_split = 0.15 , callbacks = [tensor])









Epoch 1/25
167/167 ━━━━━━━━━━━━━━━━━━━━ 51s 277ms/step - accuracy: 0.7976 - loss: 1.7946 - val_accuracy: 0.9588 - val_loss: 0.1069
Epoch 2/25
167/167 ━━━━━━━━━━━━━━━━━━━━ 45s 271ms/step - accuracy: 0.9612 - loss: 0.1112 - val_accuracy: 0.9713 - val_loss: 0.0782
Epoch 3/25
118/167 ━━━━━━━━━━━━━━━━━━━━ 12s 262ms/step - accuracy: 0.9339 - loss: 0.1652

KeyboardInterrupt: 